# LangChain basics with football stats in Italy

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1rDqka8UCUheFtqWHH6jQwIgyzMYuA_sI?usp=sharing)

In [1]:
#install all proper packages
!pip install -qU langchain langchain-community langchain-core langchain-openai openai langchain_experimental tiktoken pandas

In [2]:
!pip freeze | grep "lang\|openai\|tiktoken"

google-ai-generativelanguage==0.6.4
google-cloud-language==2.13.3
langchain==0.2.2
langchain-community==0.2.2
langchain-core==0.2.4
langchain-experimental==0.0.60
langchain-openai==0.1.8
langchain-text-splitters==0.2.0
langcodes==3.4.0
langsmith==0.1.69
language_data==1.2.0
libclang==18.1.1
openai==1.31.0
tiktoken==0.7.0


## Data Download
To work with our own data, we will save the table as a CSV file and later load it using the `CSVLoader`. Theoretically, one can use `WebCrawler` instead of a custom function or include our function in a tool.

In [3]:
import requests
import bs4
import pandas as pd
import pprint

In [4]:
from google.colab import userdata
import os

In [5]:
# may include html table here
html = """

"""

In [6]:
# Function to make HTTP GET request
def get_request(url, cookies={}, headers={}):
    headers = {
      'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    }

    return requests.get(url, cookies=cookies, headers=headers)

# Function to collect data from a URL and extract the table
def collect_data(url):
    response = get_request(url)
    soup = bs4.BeautifulSoup(response.text, features="lxml")
    table = soup.find("table",  class_ = "standing-table__table")
    # if blocked by some cloudfare
    if not table:
      soup = bs4.BeautifulSoup(html, features="lxml")
      table = soup.find("table",  class_ = "standing-table__table")
    return table

# Function to convert HTML table to pandas DataFrame
def convert_html_table_to_df(html_text):
    return pd.read_html(str(html_text))[0]

In [7]:
# Root URL for the website containing holiday data
ROOT_URL = "https://www.skysports.com/serie-a-table/2022"

# Collect the data and convert it to a DataFrame
html_text = collect_data(url=ROOT_URL)
df = convert_html_table_to_df(html_text=html_text)

<ipython-input-6-3d04d0176add>:22: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  return pd.read_html(str(html_text))[0]


- Pl (Played): The number of matches the team has played so far in the season.
- W (Wins): The number of matches the team has won.
- D (Draws): The number of matches that have ended in a draw (neither team won).
- L (Losses): The number of matches the team has lost.
- F (Goals For): The total number of goals the team has scored.
- A (Goals Against): The total number of goals the team has conceded.
- GD (Goal Difference): The difference between goals scored (F) and goals conceded (A), calculated as Goals For minus Goals Against (F - A). This is often used as a tiebreaker in rankings.
- Pts (Points): The total number of points the team has earned. Typically, teams earn 3 points for a win, 1 point for a draw, and 0 points for a loss.

In [8]:
df.head()

,#,Team,Pl,W,D,L,F,A,GD,Pts,Last 6
0,1,Napoli,38,28,6,4,77,28,49,90,NaN
1,2,Lazio,38,22,8,8,60,30,30,74,NaN
2,3,Inter Milan,38,23,3,12,71,42,29,72,NaN
3,4,AC Milan,38,20,10,8,64,43,21,70,NaN
4,5,Atalanta,38,19,7,12,66,48,18,64,NaN


In [9]:
# To insure the data is read only from the file, let's add a new team
new_row = pd.DataFrame({'#':[42],'Team': ["LLMia"], 'Pl': [38], 'W': [24], 'D': [6], 'L': [8], 'F': [48], 'A': [20], 'GD': [28], 'Pts': [78]})

# Position to insert the new row (0-indexed, so 1 is actually the 2nd row)
position = 1

# Split the DataFrame into two parts
df1 = df.iloc[:position]
df2 = df.iloc[position:]

# Concatenate the first part, new row, and second part
df = pd.concat([df1, new_row, df2]).reset_index(drop=True)
df.drop(["#"], axis=1, inplace=True)

In [10]:
df

,Team,Pl,W,D,L,F,A,GD,Pts,Last 6
0,Napoli,38,28,6,4,77,28,49,90,NaN
1,LLMia,38,24,6,8,48,20,28,78,NaN
2,Lazio,38,22,8,8,60,30,30,74,NaN
3,Inter Milan,38,23,3,12,71,42,29,72,NaN
4,AC Milan,38,20,10,8,64,43,21,70,NaN
5,Atalanta,38,19,7,12,66,48,18,64,NaN
6,Roma,38,18,9,11,50,38,12,63,NaN
7,Juventus *,38,22,6,10,56,33,23,62,NaN
8,Fiorentina,38,15,11,12,53,43,10,56,NaN
9,Bologna,38,14,12,12,53,49,4,54,NaN


In [11]:
# Save the DataFrame to a CSV file
df.iloc[:-1, :].to_csv("results_22-23.csv")

## LangChain
Now, we will import several LangChain methods that we will be utilizing. For the purposes of this demo, we will begin with a straightforward approach using the `ChatOpenAI` model. To achieve this, we will load the previously saved file and create a vector index from its contents. Additionally, we will create a simple prompt and set up a memory to store the conversation history. Finally, we will configure a `RetrievalQA` chain to bring all these components together.

In [12]:
# Set OpenAI API key from Google Colab's user environment or default
def set_openai_api_key(default_key: str = "YOUR_API_KEY") -> None:
    """Set the OpenAI API key from Google Colab's user environment or use a default value."""

    os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY") or default_key


set_openai_api_key()

In [13]:
from langchain_core.prompts.chat import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain.agents import AgentExecutor
from langchain_core.output_parsers import StrOutputParser
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.agents.output_parsers.openai_functions import OpenAIFunctionsAgentOutputParser
from langchain.indexes import VectorstoreIndexCreator
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.memory import ConversationSummaryBufferMemory, ChatMessageHistory

from operator import itemgetter

In [14]:
MEMORY_KEY = "chat_history"
verbose= True

In [15]:
from langchain_openai import ChatOpenAI, OpenAI
from langchain_openai import OpenAIEmbeddings
llm = ChatOpenAI(temperature=0,model_name="gpt-4o")
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [16]:
# Create a prompt using the template
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are Luigi. You are a assistant to help answer football questions, based only only on the data provided.
            There is no mistake in the data!
            Don't use other data sources, only use given tools!
            If asked any question about football statistics from a DataFrame regading Serie A 2022/23 results - use the pandas_df tool!
            If asked regarding the history of italian football - use the search_for_football_history tool!
            Ignore all other questions not related to Italian football!
            Only use the output of tools, don't invent!
            Here is the soccer league table with explanations for each column:
            Pl (Played): The number of matches the team has played so far in the season.
            W (Wins): The number of matches the team has won.
            D (Draws): The number of matches that have ended in a draw (neither team won).
            L (Losses): The number of matches the team has lost.
            F (Goals For): The total number of goals the team has scored.
            A (Goals Against): The total number of goals the team has conceded.
            GD (Goal Difference): The difference between goals scored (F) and goals conceded (A), calculated as Goals For minus Goals Against (F - A). This is often used as a tiebreaker in rankings.
            Pts (Points): The total number of points the team has earned. Typically, teams earn 3 points for a win, 1 point for a draw, and 0 points for a loss.
            """,
        ),
        MessagesPlaceholder(variable_name=MEMORY_KEY),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [17]:
# Creating memory

history = ChatMessageHistory()
history.add_user_message("Hi, my name is Ivan.")
history.add_ai_message("Hello Ivan! How can I assist you today?")

memory = ConversationSummaryBufferMemory(
    llm=llm,
    return_messages=True,
    memory_key=MEMORY_KEY,
    chat_memory=history
)

In [18]:
print(memory.load_memory_variables({}))
memory.predict_new_summary(existing_summary="", messages=history.messages)

{'chat_history': [HumanMessage(content='Hi, my name is Ivan.'), AIMessage(content='Hello Ivan! How can I assist you today?')]}


'The human introduces themselves as Ivan. The AI greets Ivan and asks how it can assist him today.'

In [19]:
import pprint
text = '''

Italian football has a storied history that dates back to the late 19th century, with the sport gaining significant popularity in the early 20th century. The formation of the Italian Football Federation in 1898 marked the beginning of organized football competitions in Italy. The 1930s were a golden era for Italian football, with the national team, known as the Azzurri, achieving remarkable success on the international stage. Under the guidance of coach Vittorio Pozzo, Italy won the FIFA World Cup twice, first in 1934 on home soil and again in 1938 in France. Key players like Luigi and Mario played crucial roles in these triumphs, displaying exceptional skill and teamwork that cemented Italy's reputation as a football powerhouse.

The post-war period saw the Italian Serie A league become one of the most competitive and prestigious in the world, attracting top talents from around the globe. Clubs like Juventus, AC Milan, and Inter Milan dominated both domestically and in European competitions. The 1980s and 1990s witnessed Italian clubs winning numerous UEFA Champions League titles, with legendary players such as Paolo Maldini, Roberto Baggio, and Alessandro Del Piero becoming household names. The tactical discipline and defensive prowess known as "Catenaccio" became synonymous with Italian football, contributing to the national team's success in reaching the World Cup final in 1994, though they were narrowly defeated by Brazil.

The 21st century brought a new era for Italian football, with the national team achieving its fourth World Cup victory in 2006. Under the management of Marcello Lippi, the Azzurri displayed resilience and tactical acumen throughout the tournament held in Germany. Key players like Fabio Cannavaro, Gianluigi Buffon, and Andrea Pirlo were instrumental in Italy's success, leading the team to a memorable final against France, which was decided by a penalty shootout after a 1-1 draw. This victory not only reestablished Italy as a dominant force in world football but also inspired a new generation of players and fans. Despite facing challenges in subsequent years, Italian football continues to produce world-class talent and remains an integral part of the country's cultural and sporting identity.

'''

pprint.pprint(text)

('\n'
 '\n'
 'Italian football has a storied history that dates back to the late 19th '
 'century, with the sport gaining significant popularity in the early 20th '
 'century. The formation of the Italian Football Federation in 1898 marked the '
 'beginning of organized football competitions in Italy. The 1930s were a '
 'golden era for Italian football, with the national team, known as the '
 'Azzurri, achieving remarkable success on the international stage. Under the '
 'guidance of coach Vittorio Pozzo, Italy won the FIFA World Cup twice, first '
 'in 1934 on home soil and again in 1938 in France. Key players like Luigi and '
 'Mario played crucial roles in these triumphs, displaying exceptional skill '
 "and teamwork that cemented Italy's reputation as a football powerhouse.\n"
 '\n'
 'The post-war period saw the Italian Serie A league become one of the most '
 'competitive and prestigious in the world, attracting top talents from around '
 'the globe. Clubs like Juventus, AC Milan

In [20]:
from langchain.schema.document import Document

# Creating vectorstore

def load_index():
    # if you want to avoid the step of saving/loading a file, you can use the `from_documents()` method of the VectorstoreIndexCreator()
    #loader = CSVLoader(file_path='results_22-23.csv')
    #index = VectorstoreIndexCreator().from_loaders([loader])
    docs = [Document(page_content=x) for x in text.split('/n')]
    index = VectorstoreIndexCreator(embedding=embeddings).from_documents(docs)
    return index

retriever=load_index().vectorstore.as_retriever()

/usr/local/lib/python3.10/dist-packages/langchain/indexes/vectorstore.py:129: UserWarning: Using InMemoryVectorStore as the default vectorstore.This memory store won't persist data. You should explicitlyspecify a vectorstore when using VectorstoreIndexCreator
  warnings.warn(


### Pandas Agent as a tool

In [21]:
# Import things that are needed generically
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool, format_tool_to_openai_function, Tool
from langchain.tools.retriever import create_retriever_tool
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents import create_csv_agent

pandas_agent = create_pandas_dataframe_agent(llm, df, verbose=True, handle_parsing_errors=True, agent_type="openai-tools", prefix="There is no mistake in the data! Use querying only!")

# Define the tool using the agent
pandas_tool = Tool(
    name="pandas_df",
    func=pandas_agent.run,
    description="A tool to query football statistics from a DataFrame regading Serie A 2022/23 results",
)

retrive_tool = create_retriever_tool(
    retriever,
    "search_for_football_history",
    "Returns information regarding the history of football in Italy",
)

tools = [pandas_tool, retrive_tool]
formatted_functions = [format_tool_to_openai_function(t) for t in tools]
llm_with_tools = llm.bind(functions=formatted_functions)

/usr/local/lib/python3.10/dist-packages/langchain_experimental/agents/agent_toolkits/pandas/base.py:249: UserWarning: Received additional kwargs {'handle_parsing_errors': True} which are no longer supported.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The function `format_tool_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 0.3.0. Use langchain_core.utils.function_calling.convert_to_openai_function() instead.
  warn_deprecated(


In [22]:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        ),
    }
    | RunnablePassthrough.assign(
        **{MEMORY_KEY: RunnableLambda(memory.load_memory_variables) | itemgetter(MEMORY_KEY)}
    )
    | prompt
    | llm_with_tools
    | OpenAIFunctionsAgentOutputParser()
)

qa = AgentExecutor(agent=agent, tools=tools, memory=memory, verbose=verbose, handle_parsing_errors=True)

## Q&A

In [23]:
# Function to print the response for a given query
def print_response_for_query(query):
    prompt_update = ' Do not change the values, keep the data as is'
    query += prompt_update
    result = qa.invoke({"input": query})['output']
    output_format = f"""===================================
    {result}
    ==================================="""
    return pprint.pprint(output_format)

### Asking questions

In [24]:
query = "Who became the runner-up of the English Premier League in 2022/23?"
print_response_for_query(query)



> Entering new AgentExecutor chain...
I can only provide information related to Italian football. If you have any questions about Serie A or the history of Italian football, feel free to ask!

> Finished chain.
('===================================\n'
 '    I can only provide information related to Italian football. If you have '
 'any questions about Serie A or the history of Italian football, feel free to '
 'ask!\n'
 '    ===================================')


Correct response. What about Serie A?

In [25]:
query = "Sorry, I meant in Serie A"
print_response_for_query(query)



> Entering new AgentExecutor chain...

Invoking: `pandas_df` with `Pl, W, D, L, F, A, GD, Pts`




> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df['Pl'].tolist()"}`


[38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38]
Invoking: `python_repl_ast` with `{'query': "df['W'].tolist()"}`


[28, 24, 22, 23, 20, 19, 18, 22, 15, 14, 14, 14, 11, 12, 10, 9, 8, 6, 7, 5, 3]
Invoking: `python_repl_ast` with `{'query': "df['D'].tolist()"}`


[6, 6, 8, 3, 10, 7, 9, 6, 11, 12, 11, 10, 13, 9, 13, 15, 12, 13, 10, 12, 10]
Invoking: `python_repl_ast` with `{'query': "df['L'].tolist()"}`


[4, 8, 8, 12, 8, 12, 11, 10, 12, 12, 13, 14, 14, 17, 15, 14, 18, 19, 21, 21, 25]
Invoking: `python_repl_ast` with `{'query': "df['F'].tolist()"}`


[77, 48, 60, 71, 64, 66, 50, 56, 53, 53, 42, 48, 47, 47, 37, 48, 33, 31, 31, 36, 24]
Invoking: `python_repl_ast` with `{'query': "df['A'].tolist()"}`


[28, 20, 30, 42, 43, 48, 38, 33, 43, 49, 41, 52, 4

Did you notice, how we used the **memory** here? If it wasn't for it, the response would've sounded as:
> Sorry, I can't understand you. What exactly do you want to know about Serie A?

In [26]:
query = "What place Lazio took?"
print_response_for_query(query)



> Entering new AgentExecutor chain...

Invoking: `pandas_df` with `Serie A 2022/23`




> Entering new AgentExecutor chain...
The Serie A 2022/23 season standings for the top 5 teams are as follows:

|    | Team        |   Pl |   W |   D |   L |   F |   A |   GD |   Pts |   Last 6 |
|---:|:------------|-----:|----:|----:|----:|----:|----:|-----:|------:|---------:|
|  0 | Napoli      |   38 |  28 |   6 |   4 |  77 |  28 |   49 |    90 |      nan |
|  1 | LLMia       |   38 |  24 |   6 |   8 |  48 |  20 |   28 |    78 |      nan |
|  2 | Lazio       |   38 |  22 |   8 |   8 |  60 |  30 |   30 |    74 |      nan |
|  3 | Inter Milan |   38 |  23 |   3 |  12 |  71 |  42 |   29 |    72 |      nan |
|  4 | AC Milan    |   38 |  20 |  10 |   8 |  64 |  43 |   21 |    70 |      nan |

- **Pl**: Matches Played
- **W**: Wins
- **D**: Draws
- **L**: Losses
- **F**: Goals For
- **A**: Goals Against
- **GD**: Goal Difference
- **Pts**: Points
- **Last 6**: Results of the last 6 matches (data not

### Multichain example

In [27]:
df.head(3)

,Team,Pl,W,D,L,F,A,GD,Pts,Last 6
0,Napoli,38,28,6,4,77,28,49,90,NaN
1,LLMia,38,24,6,8,48,20,28,78,NaN
2,Lazio,38,22,8,8,60,30,30,74,NaN


In [28]:
query = "Who scored the least amount of goals in the top 3?"
print_response_for_query(query)



> Entering new AgentExecutor chain...

Invoking: `pandas_df` with `[['Team', 'F']].head(3)`




> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df[['Team', 'F']].head(3)"}`


     Team   F
0  Napoli  77
1   LLMia  48
2   Lazio  60Here are the top 3 teams along with their 'F' (Goals For) values:

|    | Team   |   F |
|---:|:-------|----:|
|  0 | Napoli |  77 |
|  1 | LLMia  |  48 |
|  2 | Lazio  |  60 |

> Finished chain.
Here are the top 3 teams along with their 'F' (Goals For) values:

|    | Team   |   F |
|---:|:-------|----:|
|  0 | Napoli |  77 |
|  1 | LLMia  |  48 |
|  2 | Lazio  |  60 |The team that scored the least amount of goals in the top 3 is LLMia with 48 goals.

> Finished chain.
('===================================\n'
 '    The team that scored the least amount of goals in the top 3 is LLMia '
 'with 48 goals.\n'
 '    ===================================')


In [29]:
query = "How many teams scored less?"
print_response_for_query(query)



> Entering new AgentExecutor chain...

Invoking: `pandas_df` with `df[df['F'] < 48].shape[0]`




> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df[df['F'] < 48].shape[0]"}`


9There are 9 teams in the dataset that scored fewer than 48 goals.

> Finished chain.
There are 9 teams in the dataset that scored fewer than 48 goals.There are 9 teams that scored fewer than 48 goals in the Serie A 2022/23 season.

> Finished chain.
('===================================\n'
 '    There are 9 teams that scored fewer than 48 goals in the Serie A 2022/23 '
 'season.\n'
 '    ===================================')


In [30]:
df[df['F'] < 48]

,Team,Pl,W,D,L,F,A,GD,Pts,Last 6
10,Torino,38,14,11,13,42,41,1,53,NaN
12,Udinese,38,11,13,14,47,48,-1,46,NaN
13,Sassuolo,38,12,9,17,47,61,-14,45,NaN
14,Empoli,38,10,13,15,37,49,-12,43,NaN
16,Lecce,38,8,12,18,33,46,-13,36,NaN
17,Spezia,38,6,13,19,31,62,-31,31,NaN
18,Verona,38,7,10,21,31,59,-28,31,NaN
19,Cremonese,38,5,12,21,36,69,-33,27,NaN
20,Sampdoria,38,3,10,25,24,71,-47,19,NaN


### Some calculations

In [31]:
query = "What is the average goals scored for Juventus?"
print_response_for_query(query)



> Entering new AgentExecutor chain...

Invoking: `pandas_df` with `Juventus`




> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df[df['Team'] == 'Juventus']"}`


Empty DataFrame
Columns: [Team, Pl, W, D, L, F, A, GD, Pts, Last 6]
Index: []It appears that Juventus is not present in the provided dataset. If you have any other queries or need information on a different team, please let me know!

> Finished chain.
It appears that Juventus is not present in the provided dataset. If you have any other queries or need information on a different team, please let me know!It seems that Juventus is not present in the provided dataset. If you have any other questions or need information on a different team, feel free to ask!

> Finished chain.
('===================================\n'
 '    It seems that Juventus is not present in the provided dataset. If you '
 'have any other questions or need information on a different team, feel free '
 'to ask!\n'
 '    =

Notice that the team Juventus contains a * symbol, what prevents to find it among the teams. Clean data always leads to better results

In [32]:
df[df['Team'].str.contains('Juventus')]

,Team,Pl,W,D,L,F,A,GD,Pts,Last 6
7,Juventus *,38,22,6,10,56,33,23,62,NaN


### Text retrieval

In [33]:
query = "Who were the most recognized players in Italian football in the 1930s?"
print_response_for_query(query)



> Entering new AgentExecutor chain...

Invoking: `search_for_football_history` with `{'query': 'most recognized players in Italian football in the 1930s'}`


The post-war period saw the Italian Serie A league become one of the most competitive and prestigious in the world, attracting top talents from around the globe. Clubs like Juventus, AC Milan, and Inter Milan dominated both domestically and in European competitions. The 1980s and 1990s witnessed Italian clubs winning numerous UEFA Champions League titles, with legendary players such as Paolo Maldini, Roberto Baggio, and Alessandro Del Piero becoming household names. The tactical discipline and defensive prowess known as "Catenaccio" became synonymous with Italian football, contributing to the national team's success in reaching the World Cup final in 1994, though they were narrowly defeated by Brazil.

Italian football has a storied history that dates back to the late 19th century, with the sport gaining significant popularity i

In [34]:
query = "Who were the most recognized players in Italian football in the modern era?"
print_response_for_query(query)



> Entering new AgentExecutor chain...

Invoking: `search_for_football_history` with `{'query': 'most recognized players in Italian football in the modern era'}`


The post-war period saw the Italian Serie A league become one of the most competitive and prestigious in the world, attracting top talents from around the globe. Clubs like Juventus, AC Milan, and Inter Milan dominated both domestically and in European competitions. The 1980s and 1990s witnessed Italian clubs winning numerous UEFA Champions League titles, with legendary players such as Paolo Maldini, Roberto Baggio, and Alessandro Del Piero becoming household names. The tactical discipline and defensive prowess known as "Catenaccio" became synonymous with Italian football, contributing to the national team's success in reaching the World Cup final in 1994, though they were narrowly defeated by Brazil.

The 21st century brought a new era for Italian football, with the national team achieving its fourth World Cup victory in 2